In [13]:
from urllib2 import Request, urlopen, URLError
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("/home/neal/Downloads/Seattle_Police_Department_911_Incident_Response.csv")

/home/neal/anaconda/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2723: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [47]:
df["valid_date"] = df["Event Clearance Date"].isnull()

In [50]:
df["date"] = \
    df.apply(lambda x: str(x["Event Clearance Date"]).split(" ")[0] if not x["valid_date"]  else np.nan, axis=1)
df["time12"] = \
    df.apply(lambda x: str(x["Event Clearance Date"]).split(" ")[1] if not x["valid_date"]  else np.nan, axis=1)
df["meridian"] = \
    df.apply(lambda x: str(x["Event Clearance Date"]).split(" ")[2] if not x["valid_date"]  else np.nan, axis=1)

In [32]:
print "There are %i observations in this dataset" % df.shape[0]

There are 1254231 observations in this dataset


In [12]:
df.head()

,CAD CDW ID,CAD Event Number,General Offense Number,Event Clearance Code,Event Clearance Description,Event Clearance SubGroup,Event Clearance Group,Event Clearance Date,Hundred Block Location,District/Sector,Zone/Beat,Census Tract,Longitude,Latitude,Incident Location,Initial Type Description,Initial Type Subgroup,Initial Type Group,At Scene Time
0,﻿15736,10000246357,2010246357,242,FIGHT DISTURBANCE,DISTURBANCES,DISTURBANCES,07/17/2010 08:49:00 PM,3XX BLOCK OF PINE ST,M,M2,8100.2001,-122.338147,47.610975,"(47.610975163, -122.338146748)",NaN,NaN,NaN,NaN
1,15737,10000246471,2010246471,65,THEFT - MISCELLANEOUS,THEFT,OTHER PROPERTY,07/17/2010 08:50:00 PM,36XX BLOCK OF DISCOVERY PARK BLVD,Q,Q1,5700.1012,-122.404613,47.658325,"(47.658324899, -122.404612874)",NaN,NaN,NaN,NaN
2,15738,10000246255,2010246255,250,"MISCHIEF, NUISANCE COMPLAINTS","NUISANCE, MISCHIEF COMPLAINTS","NUISANCE, MISCHIEF",07/17/2010 08:55:00 PM,21XX BLOCK OF 3RD AVE,M,M2,7200.2025,-122.342843,47.613551,"(47.613551471, -122.342843234)",NaN,NaN,NaN,NaN
3,15739,10000246473,2010246473,460,TRAFFIC (MOVING) VIOLATION,TRAFFIC RELATED CALLS,TRAFFIC RELATED CALLS,07/17/2010 09:00:00 PM,7XX BLOCK OF ROY ST,D,D1,7200.1002,-122.341847,47.625401,"(47.625401388, -122.341846999)",NaN,NaN,NaN,NaN
4,15740,10000246330,2010246330,250,"MISCHIEF, NUISANCE COMPLAINTS","NUISANCE, MISCHIEF COMPLAINTS","NUISANCE, MISCHIEF",07/17/2010 09:00:00 PM,9XX BLOCK OF ALOHA ST,D,D1,6700.1009,-122.339709,47.627425,"(47.627424837, -122.339708605)",NaN,NaN,NaN,NaN


# Here are the groupings of types of calls responded to

In [13]:
pd.DataFrame(df["Event Clearance Group"].value_counts())

,Event Clearance Group
TRAFFIC RELATED CALLS,225767
SUSPICIOUS CIRCUMSTANCES,195112
DISTURBANCES,168359
LIQUOR VIOLATIONS,68723
CAR PROWL,45590
FALSE ALARMS,45358
ACCIDENT INVESTIGATION,39371
OTHER PROPERTY,36878
BURGLARY,36801
TRESPASS,33597


In [15]:
pd.DataFrame(df["Initial Type Group"].value_counts())

,Initial Type Group
SUSPICIOUS CIRCUMSTANCES,115011
TRAFFIC RELATED CALLS,81349
DISTURBANCES,71744
THEFT,69174
PARKING VIOLATIONS,56508
LIQUOR VIOLATIONS,29944
AUTO RECOVERIES,22377
MISCELLANEOUS MISDEMEANORS,22216
RESIDENTIAL BURGLARIES,19909
ASSAULTS,17301


In [ ]:
pd.to_datetime(df["Event Clearance Date"])